In [58]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('./chromedriver')

In [3]:
#Access BIS speech site
driver.get("https://www.bis.org/cbspeeches/index.htm?m=2%7C10")

In [53]:
tot_page = driver.find_elements(By.XPATH, "//div[@class='pageof']")[0]
tot_page = tot_page.find_elements_by_css_selector("*")
tot_page = int(bar2.text.split(' ')[-1].replace(',', ''))
tot_page

1601

In [59]:
#download folder
down_folder = r'..\data\bisorg_cbspeech'

In [63]:
for i in range(tot_page):    
    _datapage = driver.find_elements(By.XPATH, "//a[@data-page]")[-1]
    _html = str.encode(driver.page_source)
    _filename = 'p{0}.html.txt'.format(10000+i)
    with open(os.path.join(down_folder,_filename),'wb') as fout:
        fout.write(_html)
    _datapage.click()
    time.sleep(3)
    if i % 100 == 0:
        print('{0}th page downloaded'.format(i))
    if i==10:
        break

0th page downloaded
